# MNIST with CNN

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Helper Functions

Function to help intialize random weights for fully connected or convolutional layers, we leave the shape attribute as a parameter for this.

In [4]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

Same as init_weights, but for the biases

In [5]:
def init_bias(shape):
    #the 0.1 value was chosen at random
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

Create a 2D convolution using builtin conv2d from TF. From those docs:

Computes a 2-D convolution given 4-D `input` and `filter` tensors.

Given an input tensor of shape `[batch, in_height, in_width, in_channels]`
and a filter / kernel tensor of shape
`[filter_height, filter_width, in_channels, out_channels]`, this op
performs the following:

1. Flattens the filter to a 2-D matrix with shape
   `[filter_height * filter_width * in_channels, output_channels]`.
2. Extracts image patches from the input tensor to form a *virtual*
   tensor of shape `[batch, out_height, out_width,
   filter_height * filter_width * in_channels]`.
3. For each patch, right-multiplies the filter matrix and the image patch
   vector.


In [6]:
def conv2d(x, W):
    #x is the tensor to be 2d convoluted
    #W stands for filter not weights
    
    #dimensions
    #x -> [batch, height, width, channels]
    #W -> [filter_height, filter_width, channels_in, channels_out]
    #create a 2d convolution
    #strides is the amount that the filter moves each time
    #padding is the values added around the edge to minimise decrease in surface area
    
    #returns a tensor
    
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [7]:
def max_pool_2by2(x):
    
    #x has to be a 4d tensor
    #ksize is An int or list of ints that has length 1, 2 or 4. The size of the window for each dimension of the input tensor.
    #remember the dimensions are [batch, height, width, channels], so it makes sense that batch is 1 and channels are 1 
    #because you just want to choose a window
    
    #the words 2by2 in the function title refer to the window size of the pool
    
    #it returns a tensor
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

Using the conv2d function, we'll return an actual convolutional layer here that uses an ReLu activation.

In [1]:
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

This is a normal fully connected layer

In [9]:
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

### Placeholders

In [10]:
x = tf.placeholder(tf.float32,shape=[None,784])

In [11]:
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [1]:
from CNN_helper_functions import *

In [2]:
print(get_conv_output_dims(227, 11, 0, 4))

55.0


In [3]:
print(get_number_weights_biases(227, 11, 0, 4, 3, 96))

(34848, 96)


### Building the Layers

In [1]:
#reshape x into the dimensions required but the 2d convolution, -1 allows the first dimension to be inferred from the rest of the dimensions
#in this case 1 would take the place of 1 becuase it is one im,age in the batch
#remember the dimensions are as follows:
#x -> [batch, height, width, channels]
x_image = tf.reshape(x,[-1,28,28,1])


NameError: name 'tf' is not defined

In [13]:
# Using a 6by6 filter here, used 5by5 in video, you can play around with the filter size
# You can change the 32 output, that essentially represents the amount of filters used
# You need to pass in 32 to the next input though, the 1 comes from the original input of 
# a single image.
convo_1 = convolutional_layer(x_image,shape=[6,6,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [6]:
get_conv_output_dims(input_dim=28, kernel_dim= 6, padding=1, stride = 1 )

25.0

In [14]:
# Using a 6by6 filter here, used 5by5 in video, you can play around with the filter size
# You can actually change the 64 output if you want, you can think of that as a representation
# of the amount of 6by6 filters used.
convo_2 = convolutional_layer(convo_1_pooling,shape=[6,6,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [15]:
# Why 7 by 7 image? Because we did 2 pooling layers, so (28/2)/2 = 7
# 64 then just comes from the output of the previous Convolution
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [16]:
# NOTE THE PLACEHOLDER HERE!
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [17]:
y_pred = normal_full_layer(full_one_dropout,10)

### Loss Function

In [18]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

### Optimizer

In [19]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

### Intialize Variables

In [20]:
init = tf.global_variables_initializer()

### Session

In [21]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')

Currently on step 0
Accuracy is:
0.1368


Currently on step 100
Accuracy is:
0.8652


Currently on step 200
Accuracy is:
0.9095


Currently on step 300
Accuracy is:
0.9287


Currently on step 400
Accuracy is:
0.938


Currently on step 500
Accuracy is:
0.9471


Currently on step 600
Accuracy is:
0.9502


Currently on step 700
Accuracy is:
0.9554


Currently on step 800
Accuracy is:
0.9576


Currently on step 900
Accuracy is:
0.9606


Currently on step 1000
Accuracy is:
0.9633


Currently on step 1100
Accuracy is:
0.9657


Currently on step 1200
Accuracy is:
0.9657


Currently on step 1300
Accuracy is:
0.9678


Currently on step 1400
Accuracy is:
0.969


Currently on step 1500
Accuracy is:
0.9713


Currently on step 1600
Accuracy is:
0.9716


Currently on step 1700
Accuracy is:
0.9701


Currently on step 1800
Accuracy is:
0.9725


Currently on step 1900
Accuracy is:
0.9737


Currently on step 2000
Accuracy is:
0.9742


Currently on step 2100
Accuracy is:
0.9756


Currently on step 2200
A

## Great Job!